In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [2]:
diagnostics = pd.read_csv('./data/VehicleDiagnosticOnboardData.csv', header = None)
codes = pd.read_excel('./data/Service_Fault_Codes_1_0_0_167.xlsx')
faults = pd.read_csv('./data/J1939Faults.csv', header = None)

C:\Users\jnske\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
columns1 = ['RecordID', 'Name', 'Value', 'FaultID']
diagnostics.columns = columns1
# diagnostics = diagnostics.drop('Index', axis = 1)

In [4]:
diagnostics.head()

,RecordID,Name,Value,FaultID
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1


In [5]:
codes.head()

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...


In [6]:
columns2 = ['FaultID', 'ESS_ID', 'eventTimeStamp', 'eventDescription', 'actionDescription', 'ecuSoftwareVersion', 
            'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'SPN', 'FMI', 'Active', 'activeTransitionCount', 
            'faultValue', 'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude', 'locationTimeStamp']
faults.columns = columns2
faults.head()

,FaultID,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,FMI,Active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,locationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [7]:
faults.ecuMake.value_counts()

CMMNS              409642
unknown            294542
PACCR              225482
BNDWS               63873
PCAR                18964
VOLVO                7252
EATON                6474
?????                 686
????S                 526
????R                 327
?MMNS                 270
?CAR                  151
?ACCR                  39
?NDWS                  15
?????MX                 6
?????MX16U13D13         4
??MNS                   3
???R                    1
Name: ecuMake, dtype: int64

In [8]:
faults.loc[faults['ecuMake'].isin(['PCAR', '????R',
                                 '?CAR', '?ACCR', '???R']), 'ecuMake']="PACCR"
faults.loc[faults['ecuMake'].isin(['?MMNS', '??MNS']), 'ecuMake']="CMMNS"
faults.loc[faults['ecuMake']=='?NDWS', 'ecuMake']="BNDWS"

In [9]:
faults['EquipmentID'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
faults['EquipmentID']=faults['EquipmentID'].astype(float)
faults['EquipmentID']=faults['EquipmentID'].astype(int)

In [10]:
faults=faults.loc[(faults["ecuMake"]=="CMMNS"),:] 

In [11]:
joined_df = faults.set_index('FaultID').join(diagnostics.pivot(
    index = 'FaultID', 
    columns = 'Name', 
    values = 'Value')).reset_index().set_index('FaultID')

In [12]:
codes['Lamp Device'].value_counts()

Warning            4972
Stop / Shutdown     880
None                871
Maintenance         401
Name: Lamp Device, dtype: int64

In [42]:
codes_subset = codes[['SPN', 'J1939 FMI', 'Lamp Device', 'Cummins Description']]

In [43]:
codes_subset.columns = ['SPN', 'FMI', 'Lamp Device', 'Cummins Description']

In [44]:
codes_subset['Cummins Description'].isna().sum()

0

In [45]:
codes_subset['combo_code'] = 0
codes_subset['combo_code'] = codes_subset['SPN'].map(str) + codes_subset['FMI'].map(str)
codes_subset.head()

C:\Users\jnske\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\jnske\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SPN,FMI,Lamp Device,Cummins Description,combo_code
0,629,12,Stop / Shutdown,Engine Control Module Critical Internal Failur...,62912
1,635,7,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,6357
2,635,3,Warning,Engine Timing Actuator Driver Circuit - Voltag...,6353
3,635,4,Warning,Engine Timing Actuator Driver Circuit - Voltag...,6354
4,612,2,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,6122


In [53]:
maintenance = list(codes_subset.loc[codes_subset['Lamp Device'] == 'Maintenance'].combo_code.unique())

In [13]:
joined_df.head()

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultID,,,,,,,,,,,,,,,,,,,,,
6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,NaN,True,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67
8,990441,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,...,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
9,990442,2015-02-21 11:40:22.000,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,...,NaN,True,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,NaN,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN
11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,...,NaN,True,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69


In [14]:
joined_df['EquipmentID'].value_counts()

1814         10260
1816          8018
1601          7064
1592          6831
1815          6355
1803          6336
1809          6098
1600          5668
1603          5579
1589          5481
1559          5326
1808          5287
1804          5222
1562          5209
1818          5041
1820          4880
1806          4798
1554          4375
1573          4202
1594          4122
1563          3857
1812          3812
1569          3804
1556          3785
1564          3178
1811          3163
1805          3095
1819          3084
1590          3057
1577          3043
             ...  
105399896        2
2049             2
2069             2
2198             2
1430             2
1426             2
1354             2
2224             1
105363528        1
105355995        1
105311799        1
105349656        1
105411467        1
1428             1
2197             1
105337895        1
1672             1
1548             1
105411463        1
105322231        1
105349500        1
105340140   

In [15]:
# codes.columns.values

In [16]:
# codes_subset = codes[['SPN', 'J1939 FMI', 'Lamp Device', 'Cummins Description']]

In [17]:
# codes_subset.head()

In [18]:
# codes_subset.columns = ['SPN', 'FMI', 'Lamp Device', 'Cummins Description']

In [19]:
# codes_subset['Cummins Description'].isna().sum()

In [20]:
# codes_subset['combo_code'] = 0
# codes_subset['combo_code'] = codes_subset['SPN'].map(str) + codes_subset['FMI'].map(str)
# codes_subset.head()

In [21]:
# codes_subset['combo_code'].value_counts()

In [22]:
# codes_subset.loc[codes_subset['combo_code']=='52428631']

In [23]:
# joined_df['combo_code'] = 0
# joined_df['combo_code'] = joined_df['SPN'].map(str) + joined_df['FMI'].map(str)
# joined_df.head()

In [24]:
# joined_df.shape

In [25]:
# joined_cummins = pd.merge(joined_df, codes_subset, on = ['combo_code'], how = 'inner')

In [26]:
# joined_cummins.head()

In [27]:
# joined_cummins.shape

In [28]:
derated = joined_df.loc[((joined_df['SPN'] == 5246) & (joined_df.FMI == 0)) | ((joined_df['SPN'] == 5246) & (joined_df['FMI'] == 15)) | 
              ((joined_df['SPN'] == 5246) & (joined_df['FMI'] == 16)) | ((joined_df['SPN'] == 1569)),:]
derated.head()

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
FaultID,,,,,,,,,,,,,,,,,,,,,
41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,...,32,True,51.8,18431,False,NaN,51.97187,3276.75,0,0.58
46,990931,2015-02-21 12:10:51.000,NaN,NaN,04993120*00027849*082113134117*07700053*I0*BBZ*,79464664,6X1u10D1500000000,CMMNS,0,5246,...,NaN,True,NaN,22527,NaN,NaN,NaN,NaN,NaN,NaN
250,993542,2015-02-21 15:01:49.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,...,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN
291,994045,2015-02-21 15:35:45.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,...,32,True,91.4,18431,False,NaN,40.73865,3276.75,0,15.66
307,994237,2015-02-21 15:51:58.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,...,NaN,NaN,NaN,17407,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
derated.shape

(9223, 43)

In [29]:
joined_df['derated'] = 0

In [30]:
joined_df.loc[((joined_df['SPN'] == 5246) & (joined_df.FMI == 0)) | ((joined_df['SPN'] == 5246) & (joined_df['FMI'] == 15)) | 
              ((joined_df['SPN'] == 5246) & (joined_df['FMI'] == 16)) | ((joined_df['SPN'] == 1569)), 'derated'] = 1

In [31]:
joined_df.shape

(409915, 44)

In [46]:
joined_df['combo_code'] = 0
joined_df['combo_code'] = joined_df['SPN'].map(str) + joined_df['FMI'].map(str)
joined_df.head()

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derated,combo_code
FaultID,,,,,,,,,,,,,,,,,,,,,
6,990431,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,111.2,1023,NaN,NaN,13.6022,3276.75,NaN,6.67,0,11117
8,990441,2015-02-21 11:40:22.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,111,...,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0,11117
9,990442,2015-02-21 11:40:22.000,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,CMMNS,0,97,...,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0,9715
10,990446,2015-02-21 11:41:55.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00025921*082113134117*07700053*I0*BBZ*,79466580,6X1u10D1500000000,CMMNS,0,111,...,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0,11117
11,990459,2015-02-21 11:42:19.000,Low (Severity Low) Engine Coolant Level,NaN,04993120*00184697*060712145341*07700027*I0*BBZ*,79618845,6X1u10D1500000000,CMMNS,0,111,...,78.8,1023,NaN,NaN,64.82649,3276.75,NaN,17.69,0,11117


In [58]:
not_maintenance = joined_df.loc[~(joined_df['combo_code'].isin(maintenance))]

In [60]:
not_maintenance.combo_code.value_counts()

11118      24135
156931      8694
1113        3374
34647       2669
6419        2216
322620      1984
436418      1590
321620      1401
63012       1283
26234       1179
914         1169
36102       1136
325116      1133
32164       1086
15718       1029
963         1027
57429        988
58484        985
32512        983
57439        948
43397        943
1111         905
336231       891
969          888
64111        873
4112         831
305816       819
58489        768
32169        763
6417         734
           ...  
332716         2
54913          2
539417         2
10819          2
57454          2
265916         2
671313         2
532131         2
50973          2
54917          2
389            2
460718         2
336013         2
6553517        1
32412          1
32452          1
132611         1
57452          1
132211         1
26292          1
132811         1
8419           1
35561          1
409418         1
322612         1
43341          1
48133          1
19011         

In [62]:
# removing codes for low coolant
final_df = not_maintenance.loc[not_maintenance['combo_code'] != '11118']
final_df.combo_code.value_counts()

156931     8694
1113       3374
34647      2669
6419       2216
322620     1984
436418     1590
321620     1401
63012      1283
26234      1179
914        1169
36102      1136
325116     1133
32164      1086
15718      1029
963        1027
57429       988
58484       985
32512       983
57439       948
43397       943
1111        905
336231      891
969         888
64111       873
4112        831
305816      819
58489       768
32169       763
6417        734
8116        716
           ... 
34904         2
55692         2
53943         2
655353        2
27916         2
35834         2
574311        2
132731        2
336013        2
18816         2
1052          2
1104          2
6553518       2
57452         1
132211        1
47527         1
26292         1
19011         1
48133         1
322612        1
43341         1
409418        1
6553517       1
132611        1
8419          1
35561         1
32454         1
32412         1
32452         1
132811        1
Name: combo_code, Length

In [63]:
final_df.isnull().sum()

ESS_ID                           0
eventTimeStamp                   0
eventDescription             15238
actionDescription            77981
ecuSoftwareVersion              34
ecuSerialNumber                  0
ecuModel                         0
ecuMake                          0
ecuSource                        0
SPN                              0
FMI                              0
Active                           0
activeTransitionCount            0
faultValue                   77981
EquipmentID                      0
MCTNumber                        0
Latitude                         0
Longitude                        0
locationTimeStamp                0
AcceleratorPedal             43457
BarometricPressure           40142
CruiseControlActive          40347
CruiseControlSetSpeed        39874
DistanceLtd                  40049
EngineCoolantTemperature     40120
EngineLoad                   40285
EngineOilPressure            40110
EngineOilTemperature         40730
EngineRpm           

In [64]:
derated.EquipmentID.value_counts()

1490         408
1505         270
1790         206
1692         169
1480         165
1607         164
105344451    156
1670         156
1636         156
1888         146
1473         128
1645         128
1580         126
1383         126
1395         125
1587         109
1417         107
1711         107
1591         104
1564         100
1577          95
1494          92
1818          91
1573          84
1778          83
1608          82
1966          73
1730          70
1853          70
1787          64
            ... 
1610           2
1514           2
1994           2
1370           2
2017           2
1985           2
1945           2
1625           2
1713           2
1832           2
1914           2
2120           2
2002           2
2027           2
2019           2
1872           2
1995           2
1907           2
1859           2
1699           2
1651           2
1435           2
1944           2
105360462      2
1976           2
307            2
2015           2
105338352     

In [35]:
# joined_df.EquipmentID = joined_df.EquipmentID.astype(int)
worst_truck = joined_df.loc[joined_df.EquipmentID == 1490]

In [36]:
worst_truck.shape

(883, 44)

In [37]:
worst_truck.head()

,ESS_ID,eventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,SPN,...,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,derated
FaultID,,,,,,,,,,,,,,,,,,,,,
2866,1024457,2015-02-23 14:43:04.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,True,95,17407,NaN,NaN,66.97216,3276.75,NaN,17.11,0
4010,1045409,2015-02-24 11:20:30.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0
4362,1050117,2015-02-24 14:52:39.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,True,93.2,17407,NaN,NaN,67.07896,3276.75,NaN,15.66,0
5490,1067152,2015-02-25 11:27:39.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,NaN,NaN,1023,NaN,NaN,NaN,NaN,NaN,NaN,0
40490,2204533,2015-05-01 15:18:01.000,NaN,NaN,04993120*00063662*040213150018*07700044*I0*BBZ*,79488009,6X1u10D1500000000,CMMNS,0,4334,...,True,113,17407,NaN,NaN,60.72932,3276.75,NaN,13.63,0


In [65]:
worst_truck.isnull().sum()

ESS_ID                         0
eventTimeStamp                 0
eventDescription             314
actionDescription            883
ecuSoftwareVersion             0
ecuSerialNumber                0
ecuModel                       0
ecuMake                        0
ecuSource                      0
SPN                            0
FMI                            0
Active                         0
activeTransitionCount          0
faultValue                   883
EquipmentID                    0
MCTNumber                      0
Latitude                       0
Longitude                      0
locationTimeStamp              0
AcceleratorPedal             445
BarometricPressure           445
CruiseControlActive          445
CruiseControlSetSpeed        445
DistanceLtd                  445
EngineCoolantTemperature     445
EngineLoad                   445
EngineOilPressure            445
EngineOilTemperature         446
EngineRpm                    445
EngineTimeLtd                445
FuelLevel 

In [39]:
worst_truck.columns

Index(['ESS_ID', 'eventTimeStamp', 'eventDescription', 'actionDescription',
       'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake',
       'ecuSource', 'SPN', 'FMI', 'Active', 'activeTransitionCount',
       'faultValue', 'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude',
       'locationTimeStamp', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus',
       'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure', 'derated'],
      dtype='object')

In [40]:
features = worst_truck.loc[:, ['Latitude', 'Longitude',
       'locationTimeStamp', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus',
       'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure', 'derated']]
labels = ['Latitude', 'Longitude',
       'locationTimeStamp', 'AcceleratorPedal', 'BarometricPressure',
       'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd',
       'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure',
       'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel',
       'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus',
       'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake',
       'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle',
       'TurboBoostPressure', 'derated']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

ValueError: Found input variables with inconsistent numbers of samples: [883, 28]